In [1]:
import tensorflow as tf
import torch
import numpy as np
from numpy import abs
import datetime
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import boston_housing
import time

In [2]:
#Подгружаем данные
(x_train, y_train), (x_test, y_test) = boston_housing.load_data()

#Поскольку набор данных большой, нормализуем его весь, не попакетно
mean_label = y_train.mean(axis=0)
std_label = y_train.std(axis=0)
mean_feat = x_train.mean(axis=0)
std_feat = x_train.std(axis=0)
x_train = (x_train-mean_feat)/std_feat
y_train = (y_train-mean_label)/std_label
x_test = (x_test-mean_feat)/std_feat

#Преобразуем в формат Pytorch

xs = torch.tensor(torch.from_numpy(x_train).float(), requires_grad=True) 
ys = torch. from_numpy(y_train).float().unsqueeze_(1)
xs_test = torch.tensor(x_test).float()
ys_test = torch.tensor(y_test).float()

C:\Users\goldd\AppData\Local\Temp\ipykernel_17892\180825044.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  xs = torch.tensor(torch.from_numpy(x_train).float(), requires_grad=True) #.unsqueeze_(.-


In [4]:
#Раздельные метрики для Tensorflow + Keras и Pytorch
def tensorflow_metric(pred, target):
    return abs((pred.numpy()-target).mean())

def keras_metric(pred, target):
    return abs((pred-target) .mean())

def pytorch_metric(pred, target):
    return (pred - target).mean().abs()

In [22]:
class SimpleLinearRegression:
    def __init__(self, initializer = 'random'):
        if initializer=='one':
            self.var = 1.
        elif initializer == 'zeros':
            self.var = 0.
        elif initializer=='random':
            self.var = tf.random.uniform(shape=[], minval = 0.,maxval = 1.) 

        self.m = tf.Variable(1., shape = tf.TensorShape(None))
        self.b = tf.Variable(self.var)

    def predict(self, x):
        return tf.reduce_sum(self.m * x, 1) + self.b

    def mse(self, true, predicted):
        return tf.reduce_mean(tf.square(true-predicted))

    def update(self, X, y, learning_rate):
        with tf.GradientTape(persistent=True) as g:
            loss = self.mse(y, self.predict(X))
        
        dy_dm = g.gradient(loss, self.m)
        dy_db = g.gradient(loss, self.b)

        self.m.assign_sub(learning_rate * dy_dm)
        self.b.assign_sub(learning_rate * dy_db)

    def train(self, X, y, learning_rate = 0.1, epochs = 50):
        if len(X.shape) == 1:
            X = tf.reshape(X,[X.shape[0],1])

        self.m.assign([self.var]*X.shape[-1])

        for i in range(epochs):
            self.update(X, y, learning_rate)

In [18]:
class LinearRegressionModel(torch.nn.Module) :

    def __init__(self):
        super(LinearRegressionModel, self).__init__()
        self.linear = torch.nn.Linear(13, 1)

    def forward(self, x):
        y_pred = self.linear(x)
        return y_pred

    def train(self, loss, epochs=500):
        for epoch in range(epochs):

            pred_y = self.forward(xs)
            lossval = loss(pred_y, ys)
            #print(‘epoch {}, Loss {}'.format(epoch, Lossval))
            lossval.backward()
            #print(pred_y, Loss)
            optimizer.step()
            optimizer.zero_grad()

def MSELoss(pred, target):
    listoflosses = (pred - target) ** 2
    return listoflosses.mean()

In [24]:
#Tensorflow - регрессия

linear_model = SimpleLinearRegression('zeros')
n=1
timediff = [0 for i in range(n)]
for j in range(1):
    start = time.process_time() # 3amepsem 6pema neped Hayanom Bcex anox
    linear_model.train(x_train, y_train, learning_rate=0.1, epochs=500);
    timediff[j] = time.process_time()-start

summ_tf = 0

for k in timediff:
    summ_tf += k

In [25]:
#Keras регрессия
model = tf.keras.Sequential([keras.layers.Dense(units=1, input_shape=[13])])
model. compile(optimizer='sgd', loss='mean_squared_error')
start = time.process_time()
model.fit(x_train, y_train, epochs=500, verbose = 0)
summ_ks = time.process_time()-start

In [27]:
#PyTorch регрессия
mod = LinearRegressionModel()
optimizer = torch.optim.SGD(mod.parameters(), lr=1e-2)
start = time.process_time()
mod. train(loss=MSELoss)
summ_pt = time.process_time()-start

In [29]:
print('Общее время - Tensorflow, Keras, Pytorch', summ_tf, summ_ks, summ_pt)
print('Метрики:', '\n', 'Tensorflow - функция потерь',
    tensorflow_metric(linear_model.predict(x_test), y_test),
    '\n', 'Keras - функция потерь',
    keras_metric(model.predict([x_test]), y_test), '\n',
    'Pytorch - функция потерь',
    pytorch_metric(mod.forward(xs_test), ys_test).item())

Общее время - Tensorflow, Keras, Pytorch 2.453125 11.78125 0.84375
4/4 [==============================] - 0s 2ms/step
Метрики: 
 Tensorflow - функция потерь 22.99819206764154 
 Keras - функция потерь 22.99923983712078 
 Pytorch - функция потерь 22.997066497802734
